In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
sc = pyspark.SparkContext(appName="HelloSpark")

In [5]:
sc

<SparkContext master=local[*] appName=HelloSpark>

In [6]:
rdd = sc.parallelize([1,2,3,4])
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()

In [8]:
spark

In [10]:
!dir %SPARK_HOME%\examples\src\main\resources

 Volume in drive C has no label.
 Volume Serial Number is 960F-CB4B

 Directory of c:\opt\spark-3.1.1-bin-hadoop2.7\examples\src\main\resources

28.03.2021  09:09    <DIR>          .
28.03.2021  09:09    <DIR>          ..
28.03.2021  09:09    <DIR>          dir1
22.02.2021  03:44               130 employees.json
22.02.2021  03:44               240 full_user.avsc
22.02.2021  03:44             5˙812 kv1.txt
22.02.2021  03:44                49 people.csv
22.02.2021  03:44                73 people.json
22.02.2021  03:44                32 people.txt
22.02.2021  03:44               185 user.avsc
22.02.2021  03:44               334 users.avro
22.02.2021  03:44               547 users.orc
22.02.2021  03:44               615 users.parquet
              10 File(s)          8˙017 bytes
               3 Dir(s)   6˙822˙748˙160 bytes free


In [12]:
spark_resources = "c:/opt/spark-3.1.1-bin-hadoop2.7/examples/src/main/resources"

In [15]:
spark

In [16]:
people = spark.read.json(spark_resources + "/people.json")

In [17]:
type(people)

pyspark.sql.dataframe.DataFrame

In [18]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [19]:
people

DataFrame[age: bigint, name: string]

In [21]:
people.cache()

DataFrame[age: bigint, name: string]

In [22]:
people.persist()

DataFrame[age: bigint, name: string]

In [23]:
people.count()

3

In [24]:
people.toPandas()

,age,name
0,NaN,Michael
1,30.0,Andy
2,19.0,Justin


In [25]:
people.name

Column<'name'>

In [26]:
people.age

Column<'age'>

In [27]:
people.columns

['age', 'name']

In [29]:
people.toJSON().collect()

['{"name":"Michael"}',
 '{"age":30,"name":"Andy"}',
 '{"age":19,"name":"Justin"}']

In [32]:
people.toPandas()

,age,name
0,NaN,Michael
1,30.0,Andy
2,19.0,Justin


In [33]:
people.rdd.map(lambda person: person.name).collect()

['Michael', 'Andy', 'Justin']

In [34]:
people.where("age > 19").toPandas()

,age,name
0,30,Andy


In [36]:
select1 = people.where("age > 19").select("name")

select2 = people.select("name").where("age > 19")

In [37]:
select1.toPandas()

,name
0,Andy


In [39]:
select2.toPandas()

,name
0,Andy


In [40]:
select3 = people.select(people.name).where("age > 19")

In [41]:
select3.toPandas()

,name
0,Andy


In [42]:
type(people.name)

pyspark.sql.column.Column

In [44]:
import pyspark.sql.functions as F

In [45]:
F.col("age")

Column<'age'>

In [46]:
select4 = people.select(people.name).where(F.col("age") > 19)

In [47]:
select4.toPandas()

,name
0,Andy


In [48]:
student_data = {
    'name' : ['Adam', 'Darek', 'Piotr'],
    'age' : [34, 30, 16],
    'city' : ['Gdańsk', 'Olsztyn', 'New york']
}

In [55]:
import pandas as pd
df_obj = pd.DataFrame(student_data, columns=['name', 'city', 'age'])
df_obj

,name,city,age
0,Adam,Gdańsk,34
1,Darek,Olsztyn,30
2,Piotr,New york,16


In [56]:
df = spark.createDataFrame(df_obj)
df

DataFrame[name: string, city: string, age: bigint]

In [57]:
df.toPandas()

,name,city,age
0,Adam,Gdańsk,34
1,Darek,Olsztyn,30
2,Piotr,New york,16


In [58]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- age: long (nullable = true)



In [59]:
osoby = df

In [60]:
osoby.createOrReplaceTempView("osoby")

In [61]:
osoby.printSchema()

root
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- age: long (nullable = true)



In [62]:
spark.catalog

In [63]:
spark.catalog.listTables()

[Table(name='osoby', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [64]:
osoby.createOrReplaceGlobalTempView("osoby")

In [65]:
spark.catalog.listTables()

[Table(name='osoby', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [66]:
df = spark.sql("SELECT * FROM osoby")

In [67]:
df.show()

+-----+--------+---+
| name|    city|age|
+-----+--------+---+
| Adam|  Gdańsk| 34|
|Darek| Olsztyn| 30|
|Piotr|New york| 16|
+-----+--------+---+



In [68]:
df2 = osoby.select("*")

In [69]:
df2.show()

+-----+--------+---+
| name|    city|age|
+-----+--------+---+
| Adam|  Gdańsk| 34|
|Darek| Olsztyn| 30|
|Piotr|New york| 16|
+-----+--------+---+



In [70]:
df3 = osoby.select("name", "city")
df3.show()

+-----+--------+
| name|    city|
+-----+--------+
| Adam|  Gdańsk|
|Darek| Olsztyn|
|Piotr|New york|
+-----+--------+



In [71]:
osoby.select(osoby.name, osoby.city).toPandas()

,name,city
0,Adam,Gdańsk
1,Darek,Olsztyn
2,Piotr,New york


In [72]:
F.upper

<function pyspark.sql.functions.upper(col)>

In [73]:
osoby.select(F.upper(osoby.name), osoby.city).toPandas()

,upper(name),city
0,ADAM,Gdańsk
1,DAREK,Olsztyn
2,PIOTR,New york


In [74]:
osoby.select(F.upper(osoby.name).alias("big_name"), osoby.name, osoby.city).toPandas()

,big_name,name,city
0,ADAM,Adam,Gdańsk
1,DAREK,Darek,Olsztyn
2,PIOTR,Piotr,New york


In [75]:
osoby_with_big_name = osoby.withColumn("big_name", F.upper(osoby.name))

In [76]:
osoby_with_big_name.show()

+-----+--------+---+--------+
| name|    city|age|big_name|
+-----+--------+---+--------+
| Adam|  Gdańsk| 34|    ADAM|
|Darek| Olsztyn| 30|   DAREK|
|Piotr|New york| 16|   PIOTR|
+-----+--------+---+--------+



In [82]:
osoby_with_big_small_name = osoby_with_big_name.withColumn("small_name", F.col("big_name"))

In [84]:
osoby_with_big_small_name.show()

+-----+--------+---+--------+----------+
| name|    city|age|big_name|small_name|
+-----+--------+---+--------+----------+
| Adam|  Gdańsk| 34|    ADAM|      ADAM|
|Darek| Olsztyn| 30|   DAREK|     DAREK|
|Piotr|New york| 16|   PIOTR|     PIOTR|
+-----+--------+---+--------+----------+



In [78]:
osoby_with_small_name.show()

+-----+--------+---+----------+
| name|    city|age|small_name|
+-----+--------+---+----------+
| Adam|  Gdańsk| 34|      ADAM|
|Darek| Olsztyn| 30|     DAREK|
|Piotr|New york| 16|     PIOTR|
+-----+--------+---+----------+



In [85]:
osoby_with_big_name = osoby.withColumn("big_name", F.upper(osoby.name))

In [86]:
osoby_with_big_name.select("name", "age").where(osoby_with_big_name.age > 20).show()

+-----+---+
| name|age|
+-----+---+
| Adam| 34|
|Darek| 30|
+-----+---+



In [89]:
def to_upper(string):
    return "PREFIX_" + string.upper()

In [90]:
to_upper("wartosc")

'PREFIX_WARTOSC'

In [91]:
osoby.toPandas()

,name,city,age
0,Adam,Gdańsk,34
1,Darek,Olsztyn,30
2,Piotr,New york,16


In [93]:
udf_to_upper = F.udf(to_upper)

In [94]:
udf_to_upper

<function __main__.to_upper(string)>

In [95]:
to_upper

<function __main__.to_upper(string)>

In [96]:
type(udf_to_upper)

function

In [97]:
osoby.select(osoby.name, udf_to_upper(osoby.name)).show()

+-----+--------------+
| name|to_upper(name)|
+-----+--------------+
| Adam|   PREFIX_ADAM|
|Darek|  PREFIX_DAREK|
|Piotr|  PREFIX_PIOTR|
+-----+--------------+



In [98]:
def my_function(name, city):
    return f"{name} mieszka w {city}"

udf_my_function = F.udf(my_function)

In [99]:
osoby.select(osoby.name,  udf_my_function(osoby.name, osoby.city)).show()

+-----+-----------------------+
| name|my_function(name, city)|
+-----+-----------------------+
| Adam|   Adam mieszka w Gd...|
|Darek|   Darek mieszka w O...|
|Piotr|   Piotr mieszka w N...|
+-----+-----------------------+



In [100]:
os = osoby.select(osoby.name,  udf_my_function(osoby.name, osoby.city))

In [101]:
os.show(truncate=False)

+-----+------------------------+
|name |my_function(name, city) |
+-----+------------------------+
|Adam |Adam mieszka w Gdańsk   |
|Darek|Darek mieszka w Olsztyn |
|Piotr|Piotr mieszka w New york|
+-----+------------------------+



In [102]:
os.cache()

DataFrame[name: string, my_function(name, city): string]

In [103]:
os.take(5)

[Row(name='Adam', my_function(name, city)='Adam mieszka w Gdańsk'),
 Row(name='Darek', my_function(name, city)='Darek mieszka w Olsztyn'),
 Row(name='Piotr', my_function(name, city)='Piotr mieszka w New york')]

In [104]:
os.first()

Row(name='Adam', my_function(name, city)='Adam mieszka w Gdańsk')

In [105]:
os.take(5)

[Row(name='Adam', my_function(name, city)='Adam mieszka w Gdańsk'),
 Row(name='Darek', my_function(name, city)='Darek mieszka w Olsztyn'),
 Row(name='Piotr', my_function(name, city)='Piotr mieszka w New york')]

In [107]:
os.tail(1)

[Row(name='Piotr', my_function(name, city)='Piotr mieszka w New york')]

In [108]:
os_alias = osoby.select(osoby.name,  udf_my_function(osoby.name, osoby.city).alias("name_city"))

In [109]:
os_alias.toPandas()

,name,name_city
0,Adam,Adam mieszka w Gdańsk
1,Darek,Darek mieszka w Olsztyn
2,Piotr,Piotr mieszka w New york


In [110]:
os_alias.createOrReplaceTempView("osoby2")

In [111]:
sel = spark.sql("select * from osoby2 where name like '%a%'")

In [112]:
sel.show()

+-----+--------------------+
| name|           name_city|
+-----+--------------------+
| Adam|Adam mieszka w Gd...|
|Darek|Darek mieszka w O...|
+-----+--------------------+



In [116]:
from pyspark.sql.types import StringType
spark.udf.register("my_function", my_function, StringType())

<function __main__.my_function(name, city)>

In [117]:
spark.catalog.listTables()

[Table(name='osoby', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='osoby2', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [119]:
os_df = spark.sql("""

select name, my_function(name, city) as name_city
from osoby
where name like '%a%'

""")
os_df

DataFrame[name: string, name_city: string]

In [121]:
os_df.show()

+-----+--------------------+
| name|           name_city|
+-----+--------------------+
| Adam|Adam mieszka w Gd...|
|Darek|Darek mieszka w O...|
+-----+--------------------+



In [122]:
def strlen(s):
    return len(s)

In [124]:
from pyspark.sql.types import IntegerType
spark.udf.register("strlen", strlen, IntegerType())

<function __main__.strlen(s)>

In [126]:
os2_df = spark.sql("""

select name, my_function(name, city) as name_city, strlen(my_function(name, city)) as name_city_len
from osoby
where name like '%a%'

""")
os2_df

DataFrame[name: string, name_city: string, name_city_len: int]

In [127]:
os2_df.show(truncate=False)

+-----+-----------------------+-------------+
|name |name_city              |name_city_len|
+-----+-----------------------+-------------+
|Adam |Adam mieszka w Gdańsk  |21           |
|Darek|Darek mieszka w Olsztyn|23           |
+-----+-----------------------+-------------+



In [129]:
text = spark.read.text("c:/opt/spark-3.1.1-bin-hadoop2.7/README.md")

In [130]:
text.count()

108

In [132]:
text.select("*").toPandas()

,value
0,# Apache Spark
1,
2,Spark is a unified analytics engine for large-...
3,"high-level APIs in Scala, Java, Python, and R,..."
4,supports general computation graphs for data a...
...,...
103,
104,## Contributing
105,
106,Please review the [Contribution to Spark guide...


In [133]:
text.cache()

DataFrame[value: string]

In [134]:
not_empty = text.select("value", F.length(text.value)).where(F.length(text.value) > 0)

In [135]:
not_empty.toPandas()

,value,length(value)
0,# Apache Spark,14
1,Spark is a unified analytics engine for large-...,80
2,"high-level APIs in Scala, Java, Python, and R,...",75
3,supports general computation graphs for data a...,73
4,rich set of higher-level tools including Spark...,74
...,...,...
62,Please refer to the [Configuration Guide](http...,98
63,in the online documentation for an overview on...,70
64,## Contributing,15
65,Please review the [Contribution to Spark guide...,91


In [136]:
not_empty = text.select("value", F.length(text.value).alias("len")).where(F.length(text.value) > 0)

In [137]:
not_empty.toPandas()

,value,len
0,# Apache Spark,14
1,Spark is a unified analytics engine for large-...,80
2,"high-level APIs in Scala, Java, Python, and R,...",75
3,supports general computation graphs for data a...,73
4,rich set of higher-level tools including Spark...,74
...,...,...
62,Please refer to the [Configuration Guide](http...,98
63,in the online documentation for an overview on...,70
64,## Contributing,15
65,Please review the [Contribution to Spark guide...,91


In [138]:
not_empty = text.select("value", F.length(text.value).alias("len")).where("len > 0")

In [139]:
not_empty.toPandas()

,value,len
0,# Apache Spark,14
1,Spark is a unified analytics engine for large-...,80
2,"high-level APIs in Scala, Java, Python, and R,...",75
3,supports general computation graphs for data a...,73
4,rich set of higher-level tools including Spark...,74
...,...,...
62,Please refer to the [Configuration Guide](http...,98
63,in the online documentation for an overview on...,70
64,## Contributing,15
65,Please review the [Contribution to Spark guide...,91


In [141]:
tlen = F.length(text.value).alias("len")
not_empty = text.select(text.value, tlen).where(tlen > 0)
not_empty.toPandas()

,value,len
0,# Apache Spark,14
1,Spark is a unified analytics engine for large-...,80
2,"high-level APIs in Scala, Java, Python, and R,...",75
3,supports general computation graphs for data a...,73
4,rich set of higher-level tools including Spark...,74
...,...,...
62,Please refer to the [Configuration Guide](http...,98
63,in the online documentation for an overview on...,70
64,## Contributing,15
65,Please review the [Contribution to Spark guide...,91


In [151]:
text_split = not_empty.select(F.split(not_empty.value, " ").alias("split"))

In [152]:
text_split.show()

+--------------------+
|               split|
+--------------------+
|  [#, Apache, Spark]|
|[Spark, is, a, un...|
|[high-level, APIs...|
|[supports, genera...|
|[rich, set, of, h...|
|[MLlib, for, mach...|
|[and, Structured,...|
|[<https://spark.a...|
|[[![Jenkins, Buil...|
|[[![AppVeyor, Bui...|
|[[![PySpark, Cove...|
|[##, Online, Docu...|
|[You, can, find, ...|
|[guide,, on, the,...|
|[This, README, fi...|
|[##, Building, Sp...|
|[Spark, is, built...|
|[To, build, Spark...|
|[, , , , ./build/...|
|[(You, do, not, n...|
+--------------------+
only showing top 20 rows



In [158]:
words = text_split.select(F.explode(text_split.split).alias("word"))

In [159]:
words.show()

+-----------+
|       word|
+-----------+
|          #|
|     Apache|
|      Spark|
|      Spark|
|         is|
|          a|
|    unified|
|  analytics|
|     engine|
|        for|
|large-scale|
|       data|
|processing.|
|         It|
|   provides|
| high-level|
|       APIs|
|         in|
|     Scala,|
|      Java,|
+-----------+
only showing top 20 rows



In [156]:
words.count()

527

In [157]:
words.distinct().count()

289

In [161]:
grouped = words.groupBy("word")

In [162]:
grouped.count()

DataFrame[word: string, count: bigint]

In [163]:
grouped.count().show()

+---------------+-----+
|           word|count|
+---------------+-----+
|     [![PySpark|    1|
|         online|    1|
|         graphs|    1|
|     ["Building|    1|
|  documentation|    3|
|       command,|    2|
|    abbreviated|    1|
|       overview|    1|
|           rich|    1|
|            set|    2|
|    -DskipTests|    1|
| 1,000,000,000:|    2|
|           name|    1|
|   ["Specifying|    1|
|         stream|    1|
|           run:|    1|
|            not|    1|
|       programs|    2|
|          tests|    2|
|./dev/run-tests|    1|
+---------------+-----+
only showing top 20 rows

